#### This module generates data for the model

In [1]:
%load_ext memory_profiler
%memit
# Import necessary modules

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import json
import csv
import scrape_data as sc
import subprocess
import time

peak memory: 62.85 MiB, increment: 0.20 MiB


In [2]:

%memit
start_time = time.time()

def extract_paragraphs(url):
    # Extract the HTML content of the web page from the specified URL
    body = sc.extract_url_body(url)
    
    # Find all <p> tags in the HTML content
    paragraphs = body.find_all('p')
    
    # Define a list of lookup tokens to search for in the <p> tags
    lookup_tokens = ['ingredient', 'instruction']
    
    # Concatenate the lookup tokens into a regular expression pattern separated by '|' character
    tokens = '|'.join(token for token in lookup_tokens)
    
    # Loop over all the <p> tags found in the HTML content
    for p in paragraphs:
        # Extract the text content of the <p> tag and remove any leading/trailing white spaces
        p_text = p.text.strip()
        
        # Check if any of the lookup tokens exist in the text content of the <p> tag
        if re.findall(tokens, p_text, flags=re.IGNORECASE):
            # If any of the lookup tokens exist in the text content, append it to the neither_list
            neither_list.append(p_text)
    
# Print the time of execution
end_time =   time.time() 
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 290.65 MiB, increment: 0.00 MiB
Time elapsed: 0.00000 (seconds)


In [3]:
%memit
start_time = time.time()

loaveandlemons = pd.read_csv('loaveandlemons_dataset.csv', index_col = 0)
display(loaveandlemons.head())

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 290.68 MiB, increment: 0.00 MiB


,url,ingredients,instructions
0,https://www.loveandlemons.com/vegan-ramen/,1 recipe Mushroom Broth\n1 tablespoon rice vin...,Prepare the mushroom broth according to this r...
1,https://www.loveandlemons.com/mushroom-broth/,2 medium yellow onions\n2 tablespoons extra-vi...,"Wash and dry the onions, then remove the onion..."
2,https://www.loveandlemons.com/broccolini/,2 tablespoons extra-virgin olive oil\n3 garlic...,Heat the olive oil in a large lidded skillet o...
3,https://www.loveandlemons.com/pasta-fagioli/,"2 tablespoons extra-virgin olive oil, plus mor...",Heat the olive oil in a large pot or Dutch ove...
4,https://www.loveandlemons.com/vegan-meatballs/,16 ounces mixed cremini and shiitake mushrooms...,Preheat the oven to 425°F and line a baking sh...


Time elapsed: 0.06055 (seconds)


In [4]:

%memit
start_time = time.time()

# Replace NaN values with empty strings
loaveandlemons.fillna('', inplace=True)

# Generate lists from the data
url_list = loaveandlemons['url'].to_list()
ingredients_list = loaveandlemons['ingredients'].to_list()
instructions_list = loaveandlemons['instructions'].to_list()

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 296.42 MiB, increment: 0.00 MiB
Time elapsed: 0.00299 (seconds)


In [5]:
# Generate neither list for the model training, the instructions and ingridients in loaveandlemons site are inside divs so they wont be included
%memit
start_time = time.time()

neither_list = []
for url in url_list:
    extract_paragraphs(url)

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')    

peak memory: 296.45 MiB, increment: 0.00 MiB
Time elapsed: 957.40200 (seconds)


In [6]:
# The data was found in https://www.kaggle.com/datasets/hugodarwood/epirecipes/code?select=full_format_recipes.json
%memit
start_time = time.time()

df = pd.read_json('full_format_recipes.json')
display(df.head(2))
# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 322.43 MiB, increment: 0.00 MiB


,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0


Time elapsed: 1.42749 (seconds)


In [7]:
# Generate lists from the data
%memit
start_time = time.time()

ingredients_list_2 = df['ingredients'].to_list()
instructions_list_2 = df['directions'].to_list()
neither_list_2 =  df['desc'].dropna().tolist()

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 405.84 MiB, increment: 0.00 MiB
Time elapsed: 0.00000 (seconds)


In [8]:

# Combune the two datasets
%memit
start_time = time.time()

combined_ingredients_list =   ingredients_list + ingredients_list_2
combined_instructions_list =   instructions_list + instructions_list_2
combined_neither_list =   neither_list + neither_list_2

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 405.85 MiB, increment: 0.00 MiB
Time elapsed: 0.00200 (seconds)


In [10]:
# Combine between the lists and add labels
%memit
start_time = time.time()

dataset = []
for i in range(len(combined_neither_list)):
    dataset.append([combined_instructions_list[i], [1, 0, 0]])
    dataset.append([combined_ingredients_list[i], [0, 1, 0]])
    dataset.append([combined_neither_list[i], [0, 0, 1]])


# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 369.73 MiB, increment: 0.00 MiB
Time elapsed: 0.09837 (seconds)


In [11]:
%memit
# Write the dataset to a TSV file using UTF-8 encoding
with open('dataset.tsv', 'w+', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(dataset)
    
# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 369.88 MiB, increment: 0.00 MiB
Time elapsed: 7.97864 (seconds)


In [13]:
# Combine between the lists and add labels for the left outside dataset without neither samples
%memit
start_time = time.time()

dataset_left_out = []
for i in range(len(combined_neither_list) +1 ,len(combined_ingredients_list)):
    dataset_left_out.append([combined_instructions_list[i], [1, 0, 0]])
    dataset_left_out.append([combined_ingredients_list[i], [0, 1, 0]])

# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 370.09 MiB, increment: 0.00 MiB
Time elapsed: 0.02892 (seconds)


In [14]:
%memit
# Write the dataset to a TSV file using UTF-8 encoding
with open('dataset_left_out.tsv', 'w+', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(dataset_left_out)
    
# Print the time of execution
end_time =   time.time()
print(f'Time elapsed: {end_time - start_time :.5f} (seconds)')

peak memory: 370.12 MiB, increment: 0.00 MiB
Time elapsed: 5.17667 (seconds)
